# Simulating the Physics of Rail Guns

## Introduction

From trebuchets to chemical weapons and eventually nuclear bombs, it seems that humans have learned how to weaponize nearly every physical phenomenon there is. It turns out that electromagnetism is no different, with the US Navy recently spending hundreds of millions of dollars on the development of weapons leveraging the magnetic force to launch projectiles at hypersonic speeds.

These devices exploit the Lorentz Force, the force that a moving charge experiences due to an external magnetic field. The force is described by the following equation:

\begin{equation}
F_{Lorentz} = q \vec{v} \times \vec{B}
\end{equation}

This information reveals the simplicity of the concept of a rail gun:

![alt text](RailGunDiagram.jpg "Title")

Current flows down the left rail, through the projectile, and up the right rail. The purpose of the parallel rails is to create a strong magnetic field in the space occupied by the projectile. Recall that the magnetic field of a straight wire carrying a current is:

\begin{equation}
\vec{B} = \frac{\mu_0}{2\pi r}(\vec{I}\times\hat{r})
\end{equation}

And that the direction of this field circulates around the wire according to the right hand rule. As is visible in the diagram, the current flowing down the left rail yield a symmetric magnetic field in the region between the rails. The current through the projectile flows to the right while under the influence of a magnetic field that only has an upward component. As described by the Lorentz force described above, this will result in a magnetic force parallel to the rails.

In principle, that is the extent of the physics that make rail guns possible.

## Magnetic Forces on the Projectile

In this section, we analyze the magnetic field created by the current in both parallel rails. This allows us to fully determine the magnetic field acting on the projectile, and thus determine the force acting on the projectile. With further development, this could lead to further analysis of the projectile's acceleration and thus performance. For the purposes of this project, however, we consider a highly simplified model in the magnetostatic case. 

To model the rails, we consider two long, straight wires of negligible thickness. The projectile is modeled as a third long, straight wire running perpendicular to both. We choose the coordinate axes so that the "left" rail carries current in the negative x direction along the x axis, aligned with the origin. The projectile carries current in the positive y direction along the y axis for a the given projectile width. The "right" rail carries current in the positive x direction parallel to the "left" rail.

The system's adjustable parameters are declared as global variables which are updated by their corresponding sliders. Functions are defined to use these parameters and return vectors representing the three currents described above.

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

dist_btwn_wires = 3
I = 1
ILeft = np.array([-I, 0, 0])
IRight = np.array([I, 0, 0])
IProj = [0,I,0]
rail_length = 4

def updateCurrent(i_):
    global ILeft, IRight, IProj
    ILeft = np.array([-i_,0,0])
    IRight = np.array([i_,0,0])
    IProj = np.array([0,i_,0])

# Left Rail
def BFieldLeft(r: np.array) -> np.array:
    reff = np.array([0,r[1],r[2]])
    rnorm = np.linalg.norm(reff)
    if rnorm == 0:
        return np.array([0,0,0])
    rhat = np.divide(reff, rnorm)
    return np.cross(ILeft, rhat) / rnorm

# Right Rail
def BFieldRight(r: np.array) -> np.array:
    reff = np.array([0,r[1]-dist_btwn_wires,r[2]])
    rnorm = np.linalg.norm(reff)
    if rnorm == 0:
        return np.array([0,0,0])
    rhat = np.divide(reff, rnorm)
    return np.cross(IRight, rhat) / rnorm

# Projectile
def BFieldProj(r: np.array) -> np.array:
    reff = np.array([r[0],0,0])
    rnorm = np.linalg.norm(reff)
    if rnorm == 0:
        return np.array([0,0,0])
    rhat = np.divide(reff, rnorm)
    return np.cross(IProj, rhat) / rnorm

Using these functions, we can visually reveal the magnetic field due to the current through both rails. In the plot below, the "left" rail carries current into the page, while the "right" rail carries current out of the page. Combined, they create a magnetic field that only has a negative z component in the plane between them.

In [8]:
def plotRailBField(d_: float, i_):
    global dist_btwn_wires
    dist_btwn_wires = d_
    updateCurrent(i_)
    
    yrange = np.linspace(-1, dist_btwn_wires+1, 10)
    zrange = np.linspace(-dist_btwn_wires/2, dist_btwn_wires/2, 10)
    y = []
    z = []
    u = []
    v = []

    for j in yrange:
        for k in zrange:
            Batjk = np.add(BFieldLeft([0,j,k]), BFieldRight([0,j,k]))
            y.append(j)
            z.append(k)
            u.append(Batjk[1])
            v.append(Batjk[2])
            
    plt.quiver(y, z, u, v)
    plt.title("Magnetic field y-z plane cross section")
    plt.xlabel("y")
    plt.ylabel("z")
    plt.show()
    
rail_dist_slider = widgets.FloatSlider(value=3.0, min=1.0, max=5.0, step=.1, description="Rail Distance:")
rail_length_slider = widgets.FloatSlider(value=3.0, min=1.0, max=5.0, step=.1, description="Rail Length:")
current_slider = widgets.FloatSlider(value=1.0, min=.1, max=5.0, step=.1, description="I:")
interact(plotRailBField, d_=rail_dist_slider, i_=current_slider)

interactive(children=(FloatSlider(value=3.0, description='Rail Distance:', max=5.0, min=1.0), FloatSlider(valu…

<function __main__.plotRailBField(d_: float, i_)>

We can use the numerical values of the magnetic field due to both rails combined with the known current running through the projectile to calculate what the force would be at many points on the projectile. 

It is important to note now that all units shown in the plots in this project are arbitrary, and most constants have been omitted for easier readability. Instead, the main goal of this project is to analyze how changing certain parameters might affect the behavior of the system. For example, try to move the sliders and see if rail distance or current is more important for maximizing the force on the projectile. 

For the same reasoning, although we have calculated the force on the projectile at many different "slices," and it would be easy to perform an integration to calculate the total force on the projectile, having this number would not be too helpful. Due to the fact our "rails" are modeled as long wires, the magnetic field at a point close to the wire approaces infinity. Thus, the actual value of the force in this model is always infinite due to how we simplified the geometry.

In [9]:
# Due to symmetry, the field on the wire only has a y component
def BFieldOnProjectile(r: float) -> np.array:
    return np.add(BFieldLeft([0,r,0]), BFieldRight([0,r,0]))

def plotProjectileInfo(d_, i_):
    global dist_btwn_wires
    dist_btwn_wires = d_
    updateCurrent(i_)
    
    slices = 100
    step = dist_btwn_wires/slices
    r = np.linspace(0, dist_btwn_wires, slices)
    b = [np.linalg.norm(BFieldOnProjectile(i)) for i in r] 

    plt.plot(r, b)
    plt.title("Magnetic field magnitude on projectile")
    plt.xlabel("r")
    plt.show()

    f = [np.linalg.norm(np.cross(BFieldOnProjectile(i), IProj)) for i in r]
    plt.plot(r, f)
    plt.title("Force on the projectile")
    plt.xlabel("r")
    plt.show()

    F_total = 0
    for f_r in f:
        F_total += f_r * step

interact(plotProjectileInfo, d_=rail_dist_slider, i_=current_slider)

interactive(children=(FloatSlider(value=3.0, description='Rail Distance:', max=5.0, min=1.0), FloatSlider(valu…

<function __main__.plotProjectileInfo(d_, i_)>

## Repulsion: Why did the rail gun fail?

I mentioned that the US Navy recently spent vast amounts of money developing rail gun technology, but that wasn't the whole truth. Since then, the Navy has suspended the development of these devices due to the technological difficulty. There are several challenges that make rail guns impractical, for example the fact that the immense current causes the rails to melt and wear extremely fast. However, a more interesting challenge from the perspective of magnetostatics is the phenomenon of repulsion.

Just as the current through the rails creates a force pushing the projectile forward, the current through the projectile should also create a force pushing both rails outward. Furthermore, both rails carry high current in stright lines but opposite directions, which also creates a force pushing the rails apart. When combined, the repulsion between the rails is speculated to be a large contributing factor in why the rail gun failed. 

First, the same functions defined above can be used to graphically show the force on each rail due to the opposing currents:

In [10]:
def plotOpposingCurrentForces(d_, l_, i_):
    global dist_btwn_wires, rail_length
    dist_btwn_wires = d_
    rail_length = l_
    updateCurrent(i_)
    
    x = []
    y = []
    u = []
    v = []

    xrange = np.linspace(rail_length, 0, 20)
    yrange = np.linspace(0, 0, 10)
    for i in xrange:
        for j in yrange:
            Fatxy = np.cross(ILeft,BFieldRight([i,j,0]))
            x.append(i)
            y.append(j)
            u.append(Fatxy[0])
            v.append(Fatxy[1])
            
    xrange = np.linspace(rail_length, 0, 20)
    yrange = np.linspace(dist_btwn_wires, dist_btwn_wires, 10)
    for i in xrange:
        for j in yrange:
            Fatxy = np.cross(IRight,BFieldLeft([i,j,0]))
            x.append(i)
            y.append(j)
            u.append(Fatxy[0])
            v.append(Fatxy[1])
            
    plt.quiver(x, y, u, v)
    plt.ylim(-1,dist_btwn_wires+1)
    plt.title("Force on rails due to opposing currents")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()
    
interact(plotOpposingCurrentForces, d_=rail_dist_slider, l_=rail_length_slider,i_=current_slider)

interactive(children=(FloatSlider(value=3.0, description='Rail Distance:', max=5.0, min=1.0), FloatSlider(valu…

<function __main__.plotOpposingCurrentForces(d_, l_, i_)>

Next, the same information can be used to show the forces due to the projectile's magnetic field acting on the current through both rails.

It is important to note that, here, we expect our results to deviate dramatically from what might actually be expected in reality. This is because we are again modeling the projectile's magnetic field with the form it would be if it was a long, straight wire. In reality, rail gun projectiles have much smaller widths compared to the length of the rails, and the current-carrying element inside the projectile will be curved so as to counteract this effect. Nevertheless, we can gain some intuition with our simplified geometry.

In [11]:
def plotProjectileRepulsionForce(d_, l_, i_):
    global dist_btwn_wires, rail_length
    dist_btwn_wires = d_
    rail_length = l_
    updateCurrent(i_)
    x = []
    y = []
    u = []
    v = []

    xrange = np.linspace(rail_length, 0, 20)
    yrange = np.linspace(0, 0, 10)
    for i in xrange:
        for j in yrange:
            Fatxy = np.cross(ILeft,BFieldProj([i,j,0]))
            x.append(i)
            y.append(j)
            u.append(Fatxy[0])
            v.append(Fatxy[1])
            
    xrange = np.linspace(rail_length, 0, 20)
    yrange = np.linspace(dist_btwn_wires, dist_btwn_wires, 10)
    for i in xrange:
        for j in yrange:
            Fatxy = np.cross(IRight,BFieldProj([i,j,0]))
            x.append(i)
            y.append(j)
            u.append(Fatxy[0])
            v.append(Fatxy[1])

    plt.quiver(x, y, u, v)
    plt.ylim(-1,dist_btwn_wires+1)
    plt.title("Force on rails due to projectile magnetic field")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()
    
interact(plotProjectileRepulsionForce, d_=rail_dist_slider, l_=rail_length_slider,i_=current_slider)

interactive(children=(FloatSlider(value=3.0, description='Rail Distance:', max=5.0, min=1.0), FloatSlider(valu…

<function __main__.plotProjectileRepulsionForce(d_, l_, i_)>

In [12]:
def calcTotalRailRepulsion(d_, l_, i_):
    global dist_btwn_wires, rail_length
    dist_btwn_wires = d_
    rail_length = l_
    updateCurrent(i_)

    x = np.linspace(rail_length, 0, 100)
    y = [np.linalg.norm(np.cross(ILeft,BFieldProj([i,0,0]))) + np.linalg.norm(np.cross(ILeft,BFieldRight([i,0,0]))) for i in x]

    plt.plot(x, y)
    plt.show()
    
interact(calcTotalRailRepulsion, d_=rail_dist_slider, l_=rail_length_slider,i_=current_slider)

interactive(children=(FloatSlider(value=3.0, description='Rail Distance:', max=5.0, min=1.0), FloatSlider(valu…

<function __main__.calcTotalRailRepulsion(d_, l_, i_)>

## Final Remarks

While the physics driving the rail gun are simple on first glance, simulating them accurately is no easy feat. The visualizations given in this project are based off of highly simplified geometry, so the values produced are really not useful for anything besides comparing to each other. A more accurate simulation of this system would likely have to discretize the current at every point into small moving charges, so that the magnetic field due to each individual current element could be summed up to create a better picture of the actual fields at play. This would require significantly more development and computing time that might not be suitable for interactive plots such as in Jupyter Notebook, but the results would likely be far more accurate.